In [1]:
# Imports
import nibabel as nib
import numpy as np
import subprocess
import glob
import os

from nilearn.image import resample_img
from skimage.measure import label
from scipy.ndimage.morphology import binary_dilation

In [2]:
# Function selecting a bounding box (with a margin of choice) around optic chiasm mask created by FreeSurfer

def centered_bb(img, x_dim=32, y_dim=32, z_dim=12):

    a = np.any(img, axis=(1, 2))
    b = np.any(img, axis=(0, 2))
    c = np.any(img, axis=(0, 1))
    
    amin, amax = np.where(a)[0][[0, -1]]
    bmin, bmax = np.where(b)[0][[0, -1]]
    cmin, cmax = np.where(c)[0][[0, -1]]
    
    return int((amin+amax)/2)-int(x_dim/2)+1,int((amin+amax)/2)+int(x_dim/2), int((bmin+bmax)/2)-int(y_dim/2)+1,int((bmin+bmax)/2)+int(y_dim/2),int((cmin+cmax)/2)-int(z_dim/2)+1,int((cmin+cmax)/2)+int(z_dim/2)

In [3]:
def extract_mask(output_dir,group, sub, t1w_image, brain_mask, chiasm_mask):
    
    # Make output directory
    output_folder=output_dir+group+'/'+sub+'/'
    os.makedirs(output_folder, exist_ok=True)
    
    # Load all files
    t1w = nib.load(t1w_image)
    brain = nib.load(brain_mask)
    chiasm = nib.load(chiasm_mask)
    
    # Resample chiasm mask to T1w image's resolution
    chiasm_resampled = resample_img(chiasm, t1w.affine, t1w.shape, 'linear')
   
    # Calculate coordinates from mask
    chiasm_data = chiasm_resampled.get_fdata()
    a_min, a_max, b_min, b_max, c_min, c_max = centered_bb(chiasm_data,32,12,32)
    
    # 1. Extract patch with optic chiasm and save it
    t1w_data = t1w.get_fdata()

    cropped = t1w.slicer[a_min:a_max+1,b_min:b_max+1,c_min:c_max+1]
    cropped.to_filename(output_folder+'chiasm.nii.gz')
    
    # 2. Crop the same patch from brain mask
    
    brain_data = brain.get_fdata()
    
    # Sphere patch  
    
    # Calculate the center and radius of the sphere
    sphere_center = ((a_min+a_max)//2,(b_min+b_max)//2,(c_min+c_max)//2)
    sphere_radius = np.int((16**2+16**2+6**2)**(0.5))

    for x in range(sphere_center[0]-sphere_radius, sphere_center[0]+sphere_radius+1):
        for y in range(sphere_center[1]-sphere_radius, sphere_center[1]+sphere_radius+1):
            for z in range(sphere_center[2]-sphere_radius, sphere_center[2]+sphere_radius+1):
                
                dist = sphere_radius - ((x-sphere_center[0])**2+(y-sphere_center[1])**2+(z-sphere_center[2])**2)**(0.5)
                if dist>0: brain_data[x,y,z] = 0  
                    
    # Square patch
    
    #brain_data[a_min:a_max+1,b_min:b_max+1,c_min:c_max+1]=0
    
    img = nib.Nifti1Image(brain_data, brain.affine)
    nib.save(img, output_folder+'sampling_distribution.nii.gz')

    # Here we just want to save the sampling distribution with proper affine
    #brain_resampled = resample_img(brain, t1w.affine, t1w.shape, 'linear')
    #nib.save(brain_resampled, output_folder+'sampling_distribution.nii.gz' )
        
    return None

In [4]:
data_dir='../../1_Data/1_Input/'
#datasets=['ABIDE','Athletes','CHIASM','COBRE','Leipzig','HCP','UoN','MCIC']
datasets=['MCIC']

In [5]:
# Iterate through all datasets and all subjects
for dataset in datasets:
    
    dataset_dir=data_dir+dataset
    print(dataset)
        
    subjects = [os.path.basename(os.path.dirname(f)) for f in glob.glob(dataset_dir+'/*/mask_optic_chiasm.nii.gz')]
    
    for subject in subjects:
        
        subject_dir = dataset_dir+'/'+subject
        
        print(subject_dir)
        
        try:
            extract_mask(data_dir,dataset,subject,subject_dir+'/t1w_1mm_iso.nii.gz',subject_dir+'/t1w_1mm_iso_brain_mask.nii.gz', subject_dir+'/mask_optic_chiasm.nii.gz')
        except:
            print('Unexpected problem!')

MCIC
../../1_Data/1_Input/MCIC/A00036128
../../1_Data/1_Input/MCIC/A00036289
../../1_Data/1_Input/MCIC/A00036292
../../1_Data/1_Input/MCIC/A00036276
../../1_Data/1_Input/MCIC/A00036326
../../1_Data/1_Input/MCIC/A00036106
../../1_Data/1_Input/MCIC/A00036127
../../1_Data/1_Input/MCIC/A00036357
../../1_Data/1_Input/MCIC/A00036123
../../1_Data/1_Input/MCIC/A00036129
../../1_Data/1_Input/MCIC/A00036254
../../1_Data/1_Input/MCIC/A00036157
../../1_Data/1_Input/MCIC/A00036309
../../1_Data/1_Input/MCIC/A00036115
../../1_Data/1_Input/MCIC/A00036297
../../1_Data/1_Input/MCIC/A00036299
../../1_Data/1_Input/MCIC/A00036308
../../1_Data/1_Input/MCIC/A00036442
../../1_Data/1_Input/MCIC/A00036462
../../1_Data/1_Input/MCIC/A00036455
../../1_Data/1_Input/MCIC/A00036131
../../1_Data/1_Input/MCIC/A00036119
../../1_Data/1_Input/MCIC/A00036388
../../1_Data/1_Input/MCIC/A00036358
../../1_Data/1_Input/MCIC/A00036107
../../1_Data/1_Input/MCIC/A00036111
../../1_Data/1_Input/MCIC/A00036370
../../1_Data/1_Input/MC